In [6]:
import pandas as pd
import typing as t

In [5]:
table = pd.read_excel('LR-lab.xlsx')
table

,index,;,type,id,=,(,),{,},",",...,предикат,список_слагаемых,слагаемое,список_множителей,множитель,список_логических_выражений,логическое_выражение,вызов_логического_выражения,параметры_вызова,список_значений
0,0,NaN,s7,s8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,s11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,r3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,r4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,153,r9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,154,NaN,s7,s8,NaN,NaN,NaN,NaN,r2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,155,NaN,NaN,NaN,NaN,NaN,r14,NaN,NaN,s134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class Stack:
    def __init__(self) -> None:
        self.stack = []
        
    def push(self, value: t.Any) -> None:
        self.stack.append(value)
        
    def pop_n(self, n: int = 1) -> t.List[t.Any]:
        return [self.stack.pop(-1) for _ in range(n)]
    
    def is_empty(self) -> bool:
        return len(self.stack) == 0
    
    def watch_head(self) -> t.Any:
        return self.stack[-1]

In [11]:
from __future__ import annotations
from abc import ABC, abstractmethod


class State(ABC):
    @abstractmethod
    def next(self) -> int:
        pass

In [12]:
class Lexer:
    def __init__(self, symbol_table: t.Dict[int, t.Dict[int, State]]) -> None:
        self.token_sequence = None
        self.symbol_table = symbol_table

    def get_current_token(self) -> str:
        return self.token_sequence[0]

    def set_token_sequence(self, token_sequence: str) -> None:
        self.token_sequence = token_sequence

    # def get_state(self, current_state_number: int) -> None:
    #     assert self.get_current_token() in self.guiding_symbols[current_state_number]

    # def accept(self) -> None:
    #     self.token_sequence = self.token_sequence[1:]

In [ ]:
class STerminalState:
    def __init__(self, rule_id: int, symbol: str, state_stack: Stack, symbol_stack: Stack, lexer: Lexer) -> None:
        self._rule_id = rule_id
        self._symbol = symbol
        self._state_stack = state_stack
        self._symbol_stack = symbol_stack
        self._lexer = lexer